# __Использованные материалы__

* [LightFM Github](https://github.com/lyst/lightfm)
* [LightFM documentation](https://making.lyst.com/lightfm/docs/quickstart.html)
* [Google recommendation systems course](https://developers.google.com/machine-learning/recommendation)
* [Recommender system using Bayesian personalized ranking](https://towardsdatascience.com/recommender-system-using-bayesian-personalized-ranking-d30e98bba0b9)
* [Learning to Rank Sketchfab Models with LightFM](https://www.ethanrosenthal.com/2016/11/07/implicit-mf-part-2/)
* [How to build a Movie Recommender System in Python using LightFm](https://towardsdatascience.com/how-to-build-a-movie-recommender-system-in-python-using-lightfm-8fa49d7cbe3b)
* [The Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset/home?select=ratings_small.csv)

# __Краткое введение__

__LightFM__ - это реализация на Python'е ряда популярных алгоритмов рекомендаций, включая эффективную реализацию BPR и WARP. Он прост в использовании, быстр (благодаря многопоточности) и дает высококачественные результаты. <br>
Существуют две основные стратегии создания рекомендательных систем: 
* __Content-based Filtering__
* __Collaborative filtering__

На практике чаще всего они используются в совокупности.<br>
<em>Далее для удобства будет использоваться термин item, который подразумевает под собой сущности, рекомендуемые системой.</em>

### __Content-based Filtering__ 
Данный подход предполагает работу с метаданными пользователя, которые собираются различными способами:
* __explicit__ - пользователь заполняет анкеты для выявление предпочтений, к примеру оценивает какой-то item по дифференцированной шкале.<br>
* __implicit__ - все действия пользователя протоколируются для выявления предпочтений, к примеру переход по ссылками, информация о компьютере пользователя и тп.<br>

### __Collaborative filtering__ 
Данный подход использует группировку пользователей и item'ов по каким-то сходствам/критериям. Будет реализоваться следующая логика "Пользователям, которым понравился item $X$, также нравились item'ы $Y$". Похожесть как правило определяется следующими методами:<br>
* __Content-based__ - на основании характеристик item'ов и пользователей.<br>
* __Transaction-based__ - на основании того, входили ли item'ы в одну транзакцию, а пользователи совершали схожие действия.<br>

### Machine-learned ranking 
В __LightFM__ представлены два классических подхода MLR'а:
* __Bayesian Personalized Ranking (BLR)__ 
* __Weighted Approximate-Rank Pairwise (WARP)__ 

### Bayesian Personalized Ranking 
Основная идея заключается в выборке и попарном сравнение положительных и отрицательных item'ов. Алгоритм в упрощенном виде можно представить следующим образом:
1. Случайным образом возьмем пользователя $u$ и item $i$, который ранее был выбран пользователем, в таком случае item $i$ будет считаться <em>положительным.</em>
2. Случайным образом возьмем item $j$, который был выбран пользователем <em>реже</em>, чем $i$ (в том числе, который пользователь никогда не выбирал), в таком случае item $j$ будет считаться <em>отрицательным.</em>
3. Вычисляем оценку $p_{ui}$ и $p_{uj}$ пользователя $u$, а также положительного item'а $i$ и отрицательного item'а $j$ соответственно.
4. Находим разницу между положительными и отрицательными оценками, как $x_{uij} = p_{ui} - p_{uj}.$ 
5. Пропускаем эту разницу через сигмоид и используем ее для вычисления веса для обновления всех параметров модели с помощью градиентного шага(SGD).

### Weighted Approximate-Rank Pairwise
Концепция данного подхода схожа с BPR, за исключением случаев, когда происходит градиентный шаг:
* В BPR градиентный шаг происходит каждый раз с разницей в качестве веса.
* WARP совершает градиентный шаг только в случае неверного предсказания (т.е. оценка отрицательного item'а больше положительного). Если предсказание было верным, то продолжаем выбирать отрицательные item'ы, пока не получим неверный прогноз или не достигнем некоторого порогового значения.

Для этих целей WARP предоставляет два гиперпараметра:
1. __Margin__ - определяет насколько ошибочным должен быть прогноз для совершения градиентного шага. 
2. __Cutoff__ - определяет сколько раз мы готовы выбирать отрицательные примеры, пытаясь получить неверное предсказание, прежде чем откажемся и перейдем к следующему пользователю.

<em>Автор статьи [Learning to Rank Sketchfab Models with LightFM](https://www.ethanrosenthal.com/2016/11/07/implicit-mf-part-2/) утверждает, что на практике вероятнее всего WARP предпочтительнее для большинства рекомендательных систем, нежели BPR.</em>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import itertools
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from scipy.sparse import coo_matrix

In [3]:
#rating_dataset = pd.read_csv('LightFM-Dataset/ratings.csv', low_memory=False)[['userId','movieId','rating']].dropna()
#rating_dataset

rating_dataset = pd.read_csv("tourist_ml.csv").drop(['Unnamed: 0','ts','aspects','org_city','user_city'],axis=1)
rating_dataset = rating_dataset.dropna()
test = pd.read_csv("test_users.csv")

In [4]:
# Убираем фильмы и пользователей с малым количеством отзывов
filter_movies = (rating_dataset['org_id'].value_counts()>5)
filter_movies = filter_movies[filter_movies].index.tolist()

filter_users = (rating_dataset['user_id'].value_counts()>5)
filter_users = filter_users[filter_users].index.tolist()

rating_dataset_filtered = rating_dataset[(rating_dataset['org_id'].isin(filter_movies)) & (rating_dataset['user_id'].isin(filter_users))]
del filter_movies, filter_users
print('Shape User-Ratings unfiltered:\t{}'.format(rating_dataset.shape))
print('Shape User-Ratings filtered:\t{}'.format(rating_dataset_filtered.shape))

Shape User-Ratings unfiltered:	(129009, 3)
Shape User-Ratings filtered:	(14700, 3)


In [5]:
rating_dataset_filtered.head(10)

,user_id,org_id,rating
4,8757574242792609967,17298320470833172098,5.0
5,15654974577179003730,17298320470833172098,4.0
7,14285448944900664538,17298320470833172098,5.0
33,12105152658854232373,13351482607452884539,5.0
35,11069843890910646151,13351482607452884539,5.0
45,10599355457902438963,13351482607452884539,5.0
48,430316536897877171,13351482607452884539,5.0
49,14162972200781047346,13351482607452884539,4.0
50,12412921146234634345,13351482607452884539,5.0
52,8552988110114386274,13351482607452884539,5.0


In [6]:
rating_dataset_filtered_shuffled = rating_dataset_filtered.sample(frac=1).reset_index(drop=True)
rating_dataset_filtered_shuffled.head(10)

,user_id,org_id,rating
0,4480388664705836035,12383317104504770478,5.0
1,9906950084234981209,12046097390037935713,5.0
2,2202175419747836582,9133246752306196616,4.0
3,13006972244157624602,6654697229156652408,5.0
4,16802718425220669434,5002407858008059043,5.0
5,12093738264124809931,13534187238651328288,5.0
6,729783763263957353,12046097390037935713,5.0
7,13928435909890617267,6406273720434949141,5.0
8,1378766229221299885,3970345774375409173,5.0
9,2299607915947807517,11017596977107312062,4.0


In [7]:
n = 1000
rating_dataset_train = rating_dataset_filtered_shuffled[:-n]
rating_dataset_test = rating_dataset_filtered_shuffled[-n:]

In [8]:
rating_dataset_train

,user_id,org_id,rating
0,4480388664705836035,12383317104504770478,5.0
1,9906950084234981209,12046097390037935713,5.0
2,2202175419747836582,9133246752306196616,4.0
3,13006972244157624602,6654697229156652408,5.0
4,16802718425220669434,5002407858008059043,5.0
...,...,...,...
13695,7827265382646728537,4473403246015165215,5.0
13696,5911873803745116534,4139051656129080881,4.0
13697,15349821504867194719,14814427257061788801,4.0
13698,8757574242792609967,15215970200910572311,4.0


In [9]:
#Создадим User-Movie-matrix
user_movie_matrix = rating_dataset_train.pivot_table(index='user_id', columns='rating', values='org_id')
print('Shape User-Movie-Matrix:\t{}'.format(user_movie_matrix.shape))
user_movie_matrix.sample(3)

Shape User-Movie-Matrix:	(2049, 2)


rating,4.0,5.0
user_id,,
7301819280750949143,NaN,8.066922e+18
5838200518413306716,7.153213e+18,9.170923e+18
9245314847965133927,NaN,1.001442e+19


In [10]:
#Создадим маппинг для пользователей и фильмов
user_id_mapping = {id:i for i, id in enumerate(rating_dataset_filtered['user_id'].unique())}
movie_id_mapping = {id:i for i, id in enumerate(rating_dataset_filtered['rating'].unique())}
#Применим его к обучающему и тренировочному набору
train_user_data = rating_dataset_train['user_id'].map(user_id_mapping)
train_movie_data = rating_dataset_train['rating'].map(movie_id_mapping)

test_user_data = rating_dataset_test['user_id'].map(user_id_mapping)
test_movie_data = rating_dataset_test['rating'].map(movie_id_mapping)

In [11]:
#Создадим разреженную матрицу рейтинга
shape = (len(user_id_mapping), len(movie_id_mapping))
train_matrix = coo_matrix((rating_dataset_train['org_id'].values, (train_user_data.astype(int), train_movie_data.astype(int))), shape=shape)
test_matrix = coo_matrix((rating_dataset_test['org_id'].values, (test_user_data.astype(int), test_movie_data.astype(int))), shape=shape)

In [12]:
#Создадим модель LightFM и обучим ем
model = LightFM(loss='warp')
%timeit model.fit(train_matrix, epochs=30, num_threads=2)

311 ms ± 26.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
k = 20
print('Train precision at k={}:\t{:.4f}'.format(k, precision_at_k(model, train_matrix, k=k).mean()))
print('Test precision at k={}:\t\t{:.4f}'.format(k, precision_at_k(model, test_matrix, k=k).mean()))

Train precision at k=20:	0.0802
Test precision at k=20:		0.0539


In [14]:
#Старая версия модели с урезанным датасетом показывала подобные результаты
k = 20
print('Train precision at k={}:\t{:.4f}'.format(k, precision_at_k(model, train_matrix, k=k).mean()))
print('Test precision at k={}:\t\t{:.4f}'.format(k, precision_at_k(model, test_matrix, k=k).mean()))

Train precision at k=20:	0.0802
Test precision at k=20:		0.0539
